In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# find max list length within list

def find_max_list(list):
    list_len = [len(i) for i in list]
    return max(list_len)

In [ ]:
# rename columns if column name is duplicated 
# useful after multiple dataframe merges

class renamer():
            def __init__(self):
                  self.d = dict()

            def __call__(self, x):
                if x not in self.d:
                    self.d[x] = 0
                    return x
                else:
                    self.d[x] += 1
                    return "%s_%d" % (x, self.d[x])

In [ ]:
# loop through each row and remove duplicates (ingnoring NaNs)

def dup_dropper(_df):
    i = 0 
    while i< len(_df):
        _df = _df.rename(columns=renamer())
        temp = _df.iloc[i]
        temp = temp[(~temp.duplicated()) | temp.isna()]
        _df.iloc[i] = temp
        i+= 1
        return _df

In [ ]:
# takes target data spread across multiple columns in a dataframe and condenses it into a smaller number of columns 

def column_cleanup(item, _df):
    item = str(item)

    #_df = _df.rename(columns=renamer())
    
    # search columns for desired (ie cell number)
    item_cols = [col for col in _df.columns if str(item) in col]

    _df = _df.reset_index(drop=True)
    _df = _df.reindex(columns = _df.columns.tolist() + ['{0}_combined'.format(item)])

    # we need to split combined items on ',' but some of the original items contain ',' which we wish to keep inplace
    # we will replace the original ',' with '&&' for now. This can be returned to original format later if desired.
    for col in item_cols:
        _df['{0}_combined'.format(item)] =  _df['{0}_combined'.format(item)].astype(str) + ','+ _df[col].astype(str).replace(',','&&')
        _df['{0}_combined'.format(item)] = _df['{0}_combined'.format(item)].replace('nan,', '')
        _df['{0}_combined'.format(item)] = _df['{0}_combined'.format(item)].replace('nan', '')  

    combined_array = np.array(_df['{0}_combined'.format(item)])

    idx = 0
    while idx < len(combined_array):
        combined_array[idx] = combined_array[idx].replace('nan,', '').replace('nan', '')
        combined_array[idx] = combined_array[idx].split(',')
        idx +=1

    _df['{0}_combined'.format(item)] = combined_array

    comb_list = np.array(_df['{0}_combined'.format(item)])

    clean_list = []
    for i in comb_list:
        temp_list = []
        for j in i:
            if j not in temp_list:
                if j != '':
                    temp_list.append(j)
                    #print(temp_list)
        clean_list.append(temp_list)

    num_items = find_max_list(clean_list)

    cell = []
    for i in range(0, (num_items)):
        cell.append(np.array([]))

    i = 0
    while i < num_items:
        j = 0
        temp_array = np.array([])
        while j < len(clean_list):
            try:
                temp_array = np.append(temp_array, clean_list[j][i])
                j += 1
            except:
                temp_array = np.append(temp_array, '')
                j += 1
        cell[i] = temp_array
        i += 1

    # drop the original columns, then replace with the combined ones

    _df = _df.drop(item_cols, axis=1)
    _df = _df.drop('{0}_combined'.format(item), axis =1)

    for i in range(0, len(cell)): 
        _df['{0}_{1}_1'.format(item, (i+1))] = cell[i]
        # return the '&&' to ',' as mentioned earlier
        _df['{0}_{1}_1'.format(item, (i+1))] = _df['{0}_{1}_1'.format(item, (i+1))].replace('&&',',')

    # get rid of any remaining null columns
    _df = _df.dropna(axis=1, how='all')

    return _df

In [ ]:
# When using the dup_dropper, values are kept the first time they appear.
# For that reason, merge dfs in decreasing order of our confidence regarding their data

# Load the dataframes and merge them

#df_1 = pd.read_csv('path/*.csv', header=0)
#df_2 = pd.read_csv('path/*.csv', header=0)
#df_3 = pd.read_csv('path/*.csv', header=0)

#df = pd.merge(df_1, 
#              df_2, 
#              left_on=['fname', 'lname'],right_on=['fname','lname'], how='left')
#df = pd.merge(df, 
#              df_3, 
#              left_on=['fname', 'lname'],right_on=['fname','lname'], how='left')

#df = dup_dropper(df)
#df = df.dropna(axis=1, how='all')
#df = df.reset_index(drop=True)

#df

In [ ]:
#columns_list = df.columns.tolist()
#columns_list.sort()
#print(columns_list)

In [ ]:
# currently have to create list manually...
# can write a function to avoid doing this manually every time, come back to this later
# idea:
#  -> list append col for col contains _x or _y
# -> strip the _x/_y
#  -> create new list
#  -> loop through stripped list appending new list if item in stripped last has not yet been appended
#  
#  potential issue: above method will likely cause problems if we have very similar column names,
#  that represent different data

#to_tidy = [ ] # list of columns to be cleaned goes here

#for colz in to_tidy:
#    #print(colz) # for debugging
#    df = column_cleanup(colz, df)
    
#df

In [ ]:
#df.to_csv('path/dataframe.csv',index=False)